# 20240921
"MODULE/08.5 ★ CPE 정리_240920" 를 그대로 COPY함




In [ ]:


def multi_lot_CPE(df_residata):
    # 'shot' 별로 데이터를 그룹화 (고유한 die_x, die_y 조합)
    grouped = df_residata.groupby(['UNIQUE_ID', 'DieX', 'DieY'])
    
    # 회귀분석 결과를 저장할 리스트
    shot_regression_results = []
    
    for (unique_id, die_x, die_y), group in grouped:
        # 독립변수와 종속변수 설정 
        # 독립변수 (shot 좌표)
        rx = group['coordinate_X']
        ry = group['coordinate_Y']  

        # 종속변수 (residual)
        Yx = group['residual_x']
        Yy = group['residual_y']

        # 독립변수 배열 구성 및 최소자승법으로 회귀 계수 구하기
        X_dx = np.vstack([np.ones(len(rx)), rx/1e6, -ry/1e6
                          ]).T
        
        X_dy = np.vstack([np.ones(len(ry)), ry/1e6, rx/1e6
                          ]).T

        coeff_dx = np.linalg.lstsq(X_dx, Yx, rcond=None)[0]
        coeff_dy = np.linalg.lstsq(X_dy, Yy, rcond=None)[0]
    
        # 회귀분석 결과를 리스트에 저장
        shot_regression_results.append({
            'UNIQUE_ID': unique_id,
            'DieX': die_x,
            'DieY': die_y,
            'RK1': coeff_dx[0],
            'RK2': coeff_dy[0],
            'RK3': coeff_dx[1],
            'RK4': coeff_dy[1],
            'RK5': coeff_dx[2],
            'RK6': coeff_dy[2]
        })
    
    # 회귀분석 결과를 새로운 DataFrame으로 변환
    df_cpe = pd.DataFrame(shot_regression_results)
    return df_cpe




def multi_lot_cpe_fitting(df_residata, df_cpe):
    # 결과를 저장할 리스트 생성
    predictions_list = []
        
    # 각 데이터 포인트에 대해 처리
    for idx, row in df_residata.iterrows():
        
        unique_id = row['UNIQUE_ID']
        die_x = row['DieX']
        die_y = row['DieY']
        coordinate_x = row['coordinate_X']
        coordinate_y = row['coordinate_Y']
        residual_x = row['residual_x']
        residual_y = row['residual_y']
        
        rx = coordinate_x
        ry = coordinate_y

        # 해당 LOT_ID, DieX, DieY에 해당하는 회귀계수 찾기
        coeffs = df_cpe[(df_cpe['UNIQUE_ID'] == unique_id) & (df_cpe['DieX'] == die_x) & (df_cpe['DieY'] == die_y)]
        
        if not coeffs.empty:
            coeffs = coeffs.iloc[0]
            # 회귀계수 추출
            RK1 = coeffs['RK1']
            RK2 = coeffs['RK2']
            RK3 = coeffs['RK3']
            RK4 = coeffs['RK4']
            RK5 = coeffs['RK5']
            RK6 = coeffs['RK6']
            
            # 예측값 계산
            cpe_pred_x = RK1 + RK3 * (rx / 1e6) + RK5 * (-ry / 1e6)
            cpe_pred_y = RK2 + RK4 * (ry / 1e6) + RK6 * (rx / 1e6)
            
            # CPE 잔차값 계산
            cpe_resi_x = residual_x - cpe_pred_x 
            cpe_resi_y = residual_y - cpe_pred_y

            # 결과 저장
            predictions_list.append({
                'cpe_pred_x': cpe_pred_x,
                'cpe_pred_y': cpe_pred_y,
                'cpe_resi_x': cpe_resi_x,
                'cpe_resi_y': cpe_resi_y,

            })
        else:
            # 해당하는 회귀계수가 없을 경우 NaN 처리
            predictions_list.append({
                'cpe_pred_x': np.nan,
                'cpe_pred_y': np.nan,
                'cpe_resi_x': np.nan,
                'cpe_resi_y': np.nan
            })
    
    # 결과를 DataFrame으로 변환
    df_predictions = pd.DataFrame(predictions_list)
    # 원본 데이터와 병합
    df_residata = pd.concat([df_residata.reset_index(drop=True), df_predictions], axis=1)
    return df_residata











############################ MULTI LOT CPE ############################################################################


# 데이터 불러오기
df_residata = pd.read_excel("output.xlsx", sheet_name='OSR_raw_fit_resi')


# CPE 계산
df_cpe = multi_lot_CPE(df_residata)

# 결과를 엑셀 파일에 저장
with pd.ExcelWriter("output.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_cpe.to_excel(writer, sheet_name='CPE', index=False)




############################ 새로운 모듈 추가: MULTI LOT CPE FITTING ###########################################

# multi_lot_cpe_fitting 함수 사용하여 예측값 계산
df_cpe_fit_res = multi_lot_cpe_fitting(df_residata, df_cpe)

# 결과를 엑셀 파일에 저장
with pd.ExcelWriter("output.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_cpe_fit_res.to_excel(writer, sheet_name='CPE_fit_res', index=False)

print("모든 작업이 완료되었습니다.")



